In [76]:
import pandas as pd
import numpy as np
import string

from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Load data

In [77]:
train = pd.DataFrame()
labels = pd.DataFrame()
#path_file = "C:/Users/Marielle/Documents/Uni Tartu/Introduction to Computational Neuroscience/traindata/traindata/A01TInstances.csv"
#path_label = "C:/Users/Marielle/Documents/Uni Tartu/Introduction to Computational Neuroscience/traindata//traindata/A01TLabels.csv"
path_file = "C:\\kool\\sissejuhatus arvutuslikku neuroteadusesse\\project\\traindata\\A01TInstances.csv"
path_label = "C:\\kool\\sissejuhatus arvutuslikku neuroteadusesse\\project\\traindata\\A01TLabels.csv"

data_list = []

for i in range(9):
    print("Path ", path_file)
    
    feature = pd.read_csv(path_file, header=None)
    train = train.append(feature)
    
    
    label = pd.read_csv(path_label, header=None)
    labels = labels.append(label)
    
    data_list.append([feature, label]) 

    path_file = path_file.replace(str(i+1), str(i+2)) 
    path_label = path_label.replace(str(i+1), str(i+2))
    
    # Print updated data dimensions
    print(train.shape)

print("Final data shapes")
print("Data: ", train.shape)
print("Labels: ", labels.shape)

Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A01TInstances.csv
(672528, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A02TInstances.csv
(1349697, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A03TInstances.csv
(2010227, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A04TInstances.csv
(2611142, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A05TInstances.csv
(3297262, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A06TInstances.csv
(3976242, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A07TInstances.csv
(4657313, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A08TInstances.csv
(5332583, 25)
Path  C:\kool\sissejuhatus arvutuslikku neuroteadusesse\project\traindata\A09TInstances.csv
(6005911, 25)
Final data shapes
Data:  (6005911, 25)
Labels: 

In [78]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-21.142578,-23.681641,-21.484375,-25.146484,-25.732422,-27.929688,-14.550781,-22.509766,-25.439453,-28.710938,...,-30.615234,-29.638672,-26.660156,-20.947266,-25.439453,-24.902344,-19.677734,31.738281,-13.671875,-43.945312
1,-21.923828,-23.925781,-24.316406,-25.341797,-26.074219,-25.244141,-16.064453,-22.753906,-28.320312,-29.248047,...,-31.054688,-30.712891,-26.123047,-22.998047,-27.099609,-26.367188,-23.339844,22.460938,-13.183594,-40.527344
2,-15.625000,-19.726562,-18.847656,-21.582031,-20.751953,-21.728516,-12.451172,-20.410156,-23.730469,-25.927734,...,-28.759766,-28.417969,-24.462891,-21.386719,-25.097656,-24.267578,-21.484375,33.691406,-8.300781,-38.574219
3,-16.699219,-13.720703,-20.361328,-20.849609,-24.169922,-22.021484,-10.986328,-16.357422,-25.585938,-24.951172,...,-27.783203,-28.857422,-25.537109,-19.335938,-24.316406,-24.511719,-17.919922,41.992188,-7.324219,-38.574219
4,-19.335938,-17.626953,-20.410156,-26.464844,-27.392578,-29.882812,-9.277344,-18.359375,-24.609375,-30.517578,...,-31.884766,-33.740234,-32.666016,-18.554688,-26.611328,-28.466797,-18.408203,37.109375,-15.136719,-49.804688


Label each row using 4 classes separatly for each trial

In [79]:
# rows considered after first labeled row, duration in label dataset is 313 
time_period = 1000

In [178]:
# label the rows for each trial separately because row_nr starts from 0 for each trial
trials_labels = []

for data in data_list:
    
    train = data[0]
    label = data[1]
    
    label_results = [0] * len(train) # make it work for every file

    # Extract the labels for each instance
    for index, row in label.iterrows():
        label = row[0]
        row_nr = row[1]
        duration = row[2]
        if(label == 769 or label == 770 or label == 771 or label == 772):
            duration = time_period
        for i in range(duration):
            current_label = label_results[row_nr + i - 1]
            if label == 769 or label == 770 or label == 771 or label == 772:
                label_results[row_nr + i - 1] = label
            elif current_label != 769 and current_label != 770 and current_label != 771 and current_label != 772:
                label_results[row_nr + i - 1] = 1

    # Merge the data
    se = pd.Series(label_results)
    train['label'] = se.values
    trials_labels.append(train)

In [179]:
# merge all trials together to one dataframe 
merge_trials = pd.DataFrame(trials_labels[0])
for i in range(8):
    merge_trials = merge_trials.append(trials_labels[i+1])

In [180]:
train = merge_trials

Filter rows

In [282]:
# eliminate other classes from data

class1 = train[train.label==769]
class2 = train[train.label==770]
class3 = train[train.label==771]
class4 = train[train.label==772]

result = class1.append(class2)
result = result.append(class3)
result = result.append(class4)
result = result.reset_index() 


In [283]:
# remove extra electrodes
df = result.drop(['index', 22, 23, 24], axis=1)

In [284]:
# should be 2592 trials: len(df)/time_period = 2592
df.shape

(2592000, 23)

Spectral Analyses

In [184]:
# 1) doing FFT for whole matrix 
#fft_df = pd.DataFrame(np.abs(np.fft.fft(df.iloc[:,0:22])))

In [285]:
# TODO this needs to be reviewed if numbers are correct 
# because in the end it gives the same number of instances for all file as for 1 file

# spectral analyses
last_index = 0
label_vector = [] # train labels
fft_array_rows = [] # train data prepared
#print(df.shape)

# 1/250 is sampling frequency and it probably should be changed for our own data
frequencies = np.fft.rfftfreq(1000, 1/250)
wanted_frequencies = frequencies[(frequencies < 15) & (frequencies > 6) ]


for i in range(int(len(df)/time_period)):
    current_matrix = df[last_index:last_index+time_period]
    #print(current_matrix.shape)
    
    fft_array_columns = []
    label_vector.append(current_matrix["label"][last_index])
    last_index = last_index + time_period

    # 2) OR doing FFT for single row 
    for i in range(22):
        fft_array_columns.append(np.fft.rfft(current_matrix[i])[0:len(wanted_frequencies)])
        
    fft_array_rows.append(fft_array_columns)
    
    #3) OR FFT for whole matrix of each trial 
    #fft_array_rows.append(np.fft.rfft(current_matrix,22)[0])

        
df_abs = np.abs(fft_array_rows) 
df_abs.shape


(2592, 22, 35)

In [286]:
wanted_frequencies

array([  6.25,   6.5 ,   6.75,   7.  ,   7.25,   7.5 ,   7.75,   8.  ,
         8.25,   8.5 ,   8.75,   9.  ,   9.25,   9.5 ,   9.75,  10.  ,
        10.25,  10.5 ,  10.75,  11.  ,  11.25,  11.5 ,  11.75,  12.  ,
        12.25,  12.5 ,  12.75,  13.  ,  13.25,  13.5 ,  13.75,  14.  ,
        14.25,  14.5 ,  14.75])

In [287]:
new_df_abs = []
for i in range(len(df_abs)):
    l = []
    for j in range(len(df_abs[i])):
        for n in range(len(df_abs[i][j])):
            l.append(df_abs[i][j][n])
    new_df_abs.append(l)

In [288]:
# 2) OR 3) create DataFrame for single FFT
tt = pd.DataFrame(new_df_abs)
tt['label'] = label_vector

In [289]:
tt = tt.dropna()
print(tt.shape)
tt.head()

(2558, 771)


,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,label
0,569.628906,1075.864977,163.602370,3783.787259,1083.481980,770.888842,2167.313989,1260.977125,563.139396,780.539024,...,413.648924,383.882630,637.322197,1230.547151,955.387311,305.790310,827.967172,454.694032,126.721170,769
1,75.683594,1536.422785,986.855554,2220.601179,2408.060484,2307.215751,1110.790592,2581.732096,1223.597789,1581.024944,...,147.594125,489.626509,527.069423,337.806706,287.855937,620.130386,298.449729,431.272684,245.155635,769
2,882.910156,3318.880130,5545.839750,6087.129454,3399.907827,2926.279512,2723.667828,1919.777447,1676.857801,2127.939311,...,399.353592,236.657936,322.182042,571.225826,993.470675,383.728974,417.034980,531.199106,478.543669,769
3,412.158203,2307.057161,2894.864187,1238.502125,1094.663464,801.676684,638.826382,2233.512717,244.855134,2566.030343,...,581.631123,208.573848,197.078957,670.544038,437.508261,838.672249,492.289315,633.668412,202.516339,769
4,1475.537109,869.259342,370.950616,1616.516830,1619.726416,1062.155907,1650.047419,49.054233,840.881685,667.118401,...,660.741811,384.557118,496.877304,242.721397,511.157590,235.829898,47.816637,235.751017,317.257777,769


Splitting 

In [290]:
data_x = pd.DataFrame(tt.iloc[:,0:tt.shape[1] - 1]) # Instances of the data
data_y = pd.DataFrame(tt.iloc[:,tt.shape[1] - 1]) # Labels of the data

In [291]:
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=0)

In [292]:
# Check on NaN values
np.any(np.isnan(X_train))

False

In [293]:
print(X_train.shape)
print(len(y_train))

(2046, 770)
2046


LDA

In [294]:
# Train the LDA model
lda_clf = LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.4)
lda_clf.fit(X_train, y_train)

C:\Users\Unknown user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Unknown user\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.4)

In [295]:
# Accuracy of prediction on train data
y_pred = lda_clf.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_train, y_pred))

Accuracy on train data  49.0713587488


In [296]:
# Accuracy of prediction on test data
y_pred = lda_clf.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_test, y_pred))

Accuracy on test data  33.3984375


In [297]:
# Precision  on test data
df = {'Precision': precision_score(y_test.values, y_pred, average=None)}
df = pd.DataFrame(data=df)
df

,Precision
0,0.294964
1,0.256410
2,0.433333
3,0.352941


In [299]:
conf_mat = confusion_matrix(y_test, y_pred, labels=[770,769,771,772])
pd.DataFrame(conf_mat)

,0,1,2,3
0,30,42,22,39
1,25,41,25,28
2,30,30,52,21
3,32,26,21,48


KNN

In [300]:
# Train the KNN model
knn = KNeighborsClassifier(n_neighbors=5, weights="uniform", algorithm="auto", leaf_size=40, p=2, metric="minkowski", metric_params=None, n_jobs=1)
knn.fit(X_train, y_train) 

C:\Users\Unknown user\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  app.launch_new_instance()


KNeighborsClassifier(algorithm='auto', leaf_size=40, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [301]:
# Accuracy of prediction on train data
y_pred = knn.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_pred, y_train ))

Accuracy on train data  50.1955034213


In [302]:
# Accuracy of prediction on test data
y_pred = knn.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_pred, y_test ))

Accuracy on test data  27.34375


SVM 

In [303]:
from sklearn import svm
clf = svm.SVC(C=1.0, kernel="poly", degree=2, gamma=5, coef0=0.0, shrinking=True, probability=True, tol=0.25, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=None)
clf.fit(X_train, y_train)  

C:\Users\Unknown user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma=5, kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.25, verbose=False)

In [304]:
# Accuracy of prediction on train data
y_pred = clf.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_pred, y_train ))

Accuracy on train data  100.0


In [305]:
# Accuracy of prediction on test data
y_pred = clf.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_pred, y_test ))

Accuracy on test data  33.984375


Naive Bayes 

In [306]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

C:\Users\Unknown user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [307]:
# Accuracy of prediction on train data
y_pred = model.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_pred, y_train ))

Accuracy on train data  31.2805474096


In [308]:
# Accuracy of prediction on train data
y_pred = model.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_pred, y_test ))

Accuracy on test data  29.1015625


Decision Tree

In [270]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy',  max_depth = 10, max_features=10, min_samples_split=10)
clf = clf.fit(X_train, y_train)
clf

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [271]:
# Accuracy of prediction on train data
y_pred = clf.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_pred, y_train ))

Accuracy on train data  52.394916911


In [272]:
# Accuracy of prediction on test data
y_pred = clf.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_pred, y_test ))

Accuracy on test data  27.5390625
